# Example batch pipeline to load GCS data into BigQuery

This notebook demonstrates using Cloud Dataflow to load data into BigQuery

## Install Dataflow

In [ ]:
!python3 -m pip install --upgrade apache-beam 'apache-beam[gcp]'

After doing the pip install (above), please restart your kernel.

## Create Beam pipeline



In [2]:
## Change these to try this notebook out
PROJECT='cloud-training-demos'
BUCKET='cloud-training-demos-ml'
REGION='us-central1'

In [5]:
import apache_beam as beam
import logging
import datetime, os

COLNAMES='UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,SCH_DEG,HCM2,MAIN,NUMBRANCH,PREDDEG,HIGHDEG,CONTROL,ST_FIPS,REGION,LOCALE,LOCALE2,LATITUDE,LONGITUDE,CCBASIC,CCUGPROF,CCSIZSET,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY,RELAFFIL,ADM_RATE,ADM_RATE_ALL,SATVR25,SATVR75,SATMT25,SATMT75,SATWR25,SATWR75,SATVRMID,SATMTMID,SATWRMID,ACTCM25,ACTCM75,ACTEN25,ACTEN75,ACTMT25,ACTMT75,ACTWR25,ACTWR75,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SAT_AVG,SAT_AVG_ALL,PCIP01,PCIP03,PCIP04,PCIP05,PCIP09,PCIP10,PCIP11,PCIP12,PCIP13,PCIP14,PCIP15,PCIP16,PCIP19,PCIP22,PCIP23,PCIP24,PCIP25,PCIP26,PCIP27,PCIP29,PCIP30,PCIP31,PCIP38,PCIP39,PCIP40,PCIP41,PCIP42,PCIP43,PCIP44,PCIP45,PCIP46,PCIP47,PCIP48,PCIP49,PCIP50,PCIP51,PCIP52,PCIP54,CIP01CERT1,CIP01CERT2,CIP01ASSOC,CIP01CERT4,CIP01BACHL,CIP03CERT1,CIP03CERT2,CIP03ASSOC,CIP03CERT4,CIP03BACHL,CIP04CERT1,CIP04CERT2,CIP04ASSOC,CIP04CERT4,CIP04BACHL,CIP05CERT1,CIP05CERT2,CIP05ASSOC,CIP05CERT4,CIP05BACHL,CIP09CERT1,CIP09CERT2,CIP09ASSOC,CIP09CERT4,CIP09BACHL,CIP10CERT1,CIP10CERT2,CIP10ASSOC,CIP10CERT4,CIP10BACHL,CIP11CERT1,CIP11CERT2,CIP11ASSOC,CIP11CERT4,CIP11BACHL,CIP12CERT1,CIP12CERT2,CIP12ASSOC,CIP12CERT4,CIP12BACHL,CIP13CERT1,CIP13CERT2,CIP13ASSOC,CIP13CERT4,CIP13BACHL,CIP14CERT1,CIP14CERT2,CIP14ASSOC,CIP14CERT4,CIP14BACHL,CIP15CERT1,CIP15CERT2,CIP15ASSOC,CIP15CERT4,CIP15BACHL,CIP16CERT1,CIP16CERT2,CIP16ASSOC,CIP16CERT4,CIP16BACHL,CIP19CERT1,CIP19CERT2,CIP19ASSOC,CIP19CERT4,CIP19BACHL,CIP22CERT1,CIP22CERT2,CIP22ASSOC,CIP22CERT4,CIP22BACHL,CIP23CERT1,CIP23CERT2,CIP23ASSOC,CIP23CERT4,CIP23BACHL,CIP24CERT1,CIP24CERT2,CIP24ASSOC,CIP24CERT4,CIP24BACHL,CIP25CERT1,CIP25CERT2,CIP25ASSOC,CIP25CERT4,CIP25BACHL,CIP26CERT1,CIP26CERT2,CIP26ASSOC,CIP26CERT4,CIP26BACHL,CIP27CERT1,CIP27CERT2,CIP27ASSOC,CIP27CERT4,CIP27BACHL,CIP29CERT1,CIP29CERT2,CIP29ASSOC,CIP29CERT4,CIP29BACHL,CIP30CERT1,CIP30CERT2,CIP30ASSOC,CIP30CERT4,CIP30BACHL,CIP31CERT1,CIP31CERT2,CIP31ASSOC,CIP31CERT4,CIP31BACHL,CIP38CERT1,CIP38CERT2,CIP38ASSOC,CIP38CERT4,CIP38BACHL,CIP39CERT1,CIP39CERT2,CIP39ASSOC,CIP39CERT4,CIP39BACHL,CIP40CERT1,CIP40CERT2,CIP40ASSOC,CIP40CERT4,CIP40BACHL,CIP41CERT1,CIP41CERT2,CIP41ASSOC,CIP41CERT4,CIP41BACHL,CIP42CERT1,CIP42CERT2,CIP42ASSOC,CIP42CERT4,CIP42BACHL,CIP43CERT1,CIP43CERT2,CIP43ASSOC,CIP43CERT4,CIP43BACHL,CIP44CERT1,CIP44CERT2,CIP44ASSOC,CIP44CERT4,CIP44BACHL,CIP45CERT1,CIP45CERT2,CIP45ASSOC,CIP45CERT4,CIP45BACHL,CIP46CERT1,CIP46CERT2,CIP46ASSOC,CIP46CERT4,CIP46BACHL,CIP47CERT1,CIP47CERT2,CIP47ASSOC,CIP47CERT4,CIP47BACHL,CIP48CERT1,CIP48CERT2,CIP48ASSOC,CIP48CERT4,CIP48BACHL,CIP49CERT1,CIP49CERT2,CIP49ASSOC,CIP49CERT4,CIP49BACHL,CIP50CERT1,CIP50CERT2,CIP50ASSOC,CIP50CERT4,CIP50BACHL,CIP51CERT1,CIP51CERT2,CIP51ASSOC,CIP51CERT4,CIP51BACHL,CIP52CERT1,CIP52CERT2,CIP52ASSOC,CIP52CERT4,CIP52BACHL,CIP54CERT1,CIP54CERT2,CIP54ASSOC,CIP54CERT4,CIP54BACHL,DISTANCEONLY,UGDS,UG,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,UGDS_WHITENH,UGDS_BLACKNH,UGDS_API,UGDS_AIANOLD,UGDS_HISPOLD,UG_NRA,UG_UNKN,UG_WHITENH,UG_BLACKNH,UG_API,UG_AIANOLD,UG_HISPOLD,PPTUG_EF,PPTUG_EF2,CURROPER,NPT4_PUB,NPT4_PRIV,NPT4_PROG,NPT4_OTHER,NPT41_PUB,NPT42_PUB,NPT43_PUB,NPT44_PUB,NPT45_PUB,NPT41_PRIV,NPT42_PRIV,NPT43_PRIV,NPT44_PRIV,NPT45_PRIV,NPT41_PROG,NPT42_PROG,NPT43_PROG,NPT44_PROG,NPT45_PROG,NPT41_OTHER,NPT42_OTHER,NPT43_OTHER,NPT44_OTHER,NPT45_OTHER,NPT4_048_PUB,NPT4_048_PRIV,NPT4_048_PROG,NPT4_048_OTHER,NPT4_3075_PUB,NPT4_3075_PRIV,NPT4_75UP_PUB,NPT4_75UP_PRIV,NPT4_3075_PROG,NPT4_3075_OTHER,NPT4_75UP_PROG,NPT4_75UP_OTHER,NUM4_PUB,NUM4_PRIV,NUM4_PROG,NUM4_OTHER,NUM41_PUB,NUM42_PUB,NUM43_PUB,NUM44_PUB,NUM45_PUB,NUM41_PRIV,NUM42_PRIV,NUM43_PRIV,NUM44_PRIV,NUM45_PRIV,NUM41_PROG,NUM42_PROG,NUM43_PROG,NUM44_PROG,NUM45_PROG,NUM41_OTHER,NUM42_OTHER,NUM43_OTHER,NUM44_OTHER,NUM45_OTHER,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,TUITFTE,INEXPFTE,AVGFACSAL,PFTFAC,PCTPELL,C150_4,C150_L4,C150_4_POOLED,C150_L4_POOLED,POOLYRS,PFTFTUG1_EF,D150_4,D150_L4,D150_4_POOLED,D150_L4_POOLED,C150_4_WHITE,C150_4_BLACK,C150_4_HISP,C150_4_ASIAN,C150_4_AIAN,C150_4_NHPI,C150_4_2MOR,C150_4_NRA,C150_4_UNKN,C150_4_WHITENH,C150_4_BLACKNH,C150_4_API,C150_4_AIANOLD,C150_4_HISPOLD,C150_L4_WHITE,C150_L4_BLACK,C150_L4_HISP,C150_L4_ASIAN,C150_L4_AIAN,C150_L4_NHPI,C150_L4_2MOR,C150_L4_NRA,C150_L4_UNKN,C150_L4_WHITENH,C150_L4_BLACKNH,C150_L4_API,C150_L4_AIANOLD,C150_L4_HISPOLD,C200_4,C200_L4,D200_4,D200_L4,RET_FT4,RET_FTL4,RET_PT4,RET_PTL4,C200_4_POOLED,C200_L4_POOLED,POOLYRS200,D200_4_POOLED,D200_L4_POOLED,PCTFLOAN,UG25ABV,CDR2,CDR3,DEATH_YR2_RT,COMP_ORIG_YR2_RT,COMP_4YR_TRANS_YR2_RT,COMP_2YR_TRANS_YR2_RT,WDRAW_ORIG_YR2_RT,WDRAW_4YR_TRANS_YR2_RT,WDRAW_2YR_TRANS_YR2_RT,ENRL_ORIG_YR2_RT,ENRL_4YR_TRANS_YR2_RT,ENRL_2YR_TRANS_YR2_RT,UNKN_ORIG_YR2_RT,UNKN_4YR_TRANS_YR2_RT,UNKN_2YR_TRANS_YR2_RT,LO_INC_DEATH_YR2_RT,LO_INC_COMP_ORIG_YR2_RT,LO_INC_COMP_4YR_TRANS_YR2_RT,LO_INC_COMP_2YR_TRANS_YR2_RT,LO_INC_WDRAW_ORIG_YR2_RT,LO_INC_WDRAW_4YR_TRANS_YR2_RT,LO_INC_WDRAW_2YR_TRANS_YR2_RT,LO_INC_ENRL_ORIG_YR2_RT,LO_INC_ENRL_4YR_TRANS_YR2_RT,LO_INC_ENRL_2YR_TRANS_YR2_RT,LO_INC_UNKN_ORIG_YR2_RT,LO_INC_UNKN_4YR_TRANS_YR2_RT,LO_INC_UNKN_2YR_TRANS_YR2_RT,MD_INC_DEATH_YR2_RT,MD_INC_COMP_ORIG_YR2_RT,MD_INC_COMP_4YR_TRANS_YR2_RT,MD_INC_COMP_2YR_TRANS_YR2_RT,MD_INC_WDRAW_ORIG_YR2_RT,MD_INC_WDRAW_4YR_TRANS_YR2_RT,MD_INC_WDRAW_2YR_TRANS_YR2_RT,MD_INC_ENRL_ORIG_YR2_RT,MD_INC_ENRL_4YR_TRANS_YR2_RT,MD_INC_ENRL_2YR_TRANS_YR2_RT,MD_INC_UNKN_ORIG_YR2_RT,MD_INC_UNKN_4YR_TRANS_YR2_RT,MD_INC_UNKN_2YR_TRANS_YR2_RT,HI_INC_DEATH_YR2_RT,HI_INC_COMP_ORIG_YR2_RT,HI_INC_COMP_4YR_TRANS_YR2_RT,HI_INC_COMP_2YR_TRANS_YR2_RT,HI_INC_WDRAW_ORIG_YR2_RT,HI_INC_WDRAW_4YR_TRANS_YR2_RT,HI_INC_WDRAW_2YR_TRANS_YR2_RT,HI_INC_ENRL_ORIG_YR2_RT,HI_INC_ENRL_4YR_TRANS_YR2_RT,HI_INC_ENRL_2YR_TRANS_YR2_RT,HI_INC_UNKN_ORIG_YR2_RT,HI_INC_UNKN_4YR_TRANS_YR2_RT,HI_INC_UNKN_2YR_TRANS_YR2_RT,DEP_DEATH_YR2_RT,DEP_COMP_ORIG_YR2_RT,DEP_COMP_4YR_TRANS_YR2_RT,DEP_COMP_2YR_TRANS_YR2_RT,DEP_WDRAW_ORIG_YR2_RT,DEP_WDRAW_4YR_TRANS_YR2_RT,DEP_WDRAW_2YR_TRANS_YR2_RT,DEP_ENRL_ORIG_YR2_RT,DEP_ENRL_4YR_TRANS_YR2_RT,DEP_ENRL_2YR_TRANS_YR2_RT,DEP_UNKN_ORIG_YR2_RT,DEP_UNKN_4YR_TRANS_YR2_RT,DEP_UNKN_2YR_TRANS_YR2_RT,IND_DEATH_YR2_RT,IND_COMP_ORIG_YR2_RT,IND_COMP_4YR_TRANS_YR2_RT,IND_COMP_2YR_TRANS_YR2_RT,IND_WDRAW_ORIG_YR2_RT,IND_WDRAW_4YR_TRANS_YR2_RT,IND_WDRAW_2YR_TRANS_YR2_RT,IND_ENRL_ORIG_YR2_RT,IND_ENRL_4YR_TRANS_YR2_RT,IND_ENRL_2YR_TRANS_YR2_RT,IND_UNKN_ORIG_YR2_RT,IND_UNKN_4YR_TRANS_YR2_RT,IND_UNKN_2YR_TRANS_YR2_RT,FEMALE_DEATH_YR2_RT,FEMALE_COMP_ORIG_YR2_RT,FEMALE_COMP_4YR_TRANS_YR2_RT,FEMALE_COMP_2YR_TRANS_YR2_RT,FEMALE_WDRAW_ORIG_YR2_RT,FEMALE_WDRAW_4YR_TRANS_YR2_RT,FEMALE_WDRAW_2YR_TRANS_YR2_RT,FEMALE_ENRL_ORIG_YR2_RT,FEMALE_ENRL_4YR_TRANS_YR2_RT,FEMALE_ENRL_2YR_TRANS_YR2_RT,FEMALE_UNKN_ORIG_YR2_RT,FEMALE_UNKN_4YR_TRANS_YR2_RT,FEMALE_UNKN_2YR_TRANS_YR2_RT,MALE_DEATH_YR2_RT,MALE_COMP_ORIG_YR2_RT,MALE_COMP_4YR_TRANS_YR2_RT,MALE_COMP_2YR_TRANS_YR2_RT,MALE_WDRAW_ORIG_YR2_RT,MALE_WDRAW_4YR_TRANS_YR2_RT,MALE_WDRAW_2YR_TRANS_YR2_RT,MALE_ENRL_ORIG_YR2_RT,MALE_ENRL_4YR_TRANS_YR2_RT,MALE_ENRL_2YR_TRANS_YR2_RT,MALE_UNKN_ORIG_YR2_RT,MALE_UNKN_4YR_TRANS_YR2_RT,MALE_UNKN_2YR_TRANS_YR2_RT,PELL_DEATH_YR2_RT,PELL_COMP_ORIG_YR2_RT,PELL_COMP_4YR_TRANS_YR2_RT,PELL_COMP_2YR_TRANS_YR2_RT,PELL_WDRAW_ORIG_YR2_RT,PELL_WDRAW_4YR_TRANS_YR2_RT,PELL_WDRAW_2YR_TRANS_YR2_RT,PELL_ENRL_ORIG_YR2_RT,PELL_ENRL_4YR_TRANS_YR2_RT,PELL_ENRL_2YR_TRANS_YR2_RT,PELL_UNKN_ORIG_YR2_RT,PELL_UNKN_4YR_TRANS_YR2_RT,PELL_UNKN_2YR_TRANS_YR2_RT,NOPELL_DEATH_YR2_RT,NOPELL_COMP_ORIG_YR2_RT,NOPELL_COMP_4YR_TRANS_YR2_RT,NOPELL_COMP_2YR_TRANS_YR2_RT,NOPELL_WDRAW_ORIG_YR2_RT,NOPELL_WDRAW_4YR_TRANS_YR2_RT,NOPELL_WDRAW_2YR_TRANS_YR2_RT,NOPELL_ENRL_ORIG_YR2_RT,NOPELL_ENRL_4YR_TRANS_YR2_RT,NOPELL_ENRL_2YR_TRANS_YR2_RT,NOPELL_UNKN_ORIG_YR2_RT,NOPELL_UNKN_4YR_TRANS_YR2_RT,NOPELL_UNKN_2YR_TRANS_YR2_RT,LOAN_DEATH_YR2_RT,LOAN_COMP_ORIG_YR2_RT,LOAN_COMP_4YR_TRANS_YR2_RT,LOAN_COMP_2YR_TRANS_YR2_RT,LOAN_WDRAW_ORIG_YR2_RT,LOAN_WDRAW_4YR_TRANS_YR2_RT,LOAN_WDRAW_2YR_TRANS_YR2_RT,LOAN_ENRL_ORIG_YR2_RT,LOAN_ENRL_4YR_TRANS_YR2_RT,LOAN_ENRL_2YR_TRANS_YR2_RT,LOAN_UNKN_ORIG_YR2_RT,LOAN_UNKN_4YR_TRANS_YR2_RT,LOAN_UNKN_2YR_TRANS_YR2_RT,NOLOAN_DEATH_YR2_RT,NOLOAN_COMP_ORIG_YR2_RT,NOLOAN_COMP_4YR_TRANS_YR2_RT,NOLOAN_COMP_2YR_TRANS_YR2_RT,NOLOAN_WDRAW_ORIG_YR2_RT,NOLOAN_WDRAW_4YR_TRANS_YR2_RT,NOLOAN_WDRAW_2YR_TRANS_YR2_RT,NOLOAN_ENRL_ORIG_YR2_RT,NOLOAN_ENRL_4YR_TRANS_YR2_RT,NOLOAN_ENRL_2YR_TRANS_YR2_RT,NOLOAN_UNKN_ORIG_YR2_RT,NOLOAN_UNKN_4YR_TRANS_YR2_RT,NOLOAN_UNKN_2YR_TRANS_YR2_RT,FIRSTGEN_DEATH_YR2_RT,FIRSTGEN_COMP_ORIG_YR2_RT,FIRSTGEN_COMP_4YR_TRANS_YR2_RT,FIRSTGEN_COMP_2YR_TRANS_YR2_RT,FIRSTGEN_WDRAW_ORIG_YR2_RT,FIRSTGEN_WDRAW_4YR_TRANS_YR2_RT,FIRSTGEN_WDRAW_2YR_TRANS_YR2_RT,FIRSTGEN_ENRL_ORIG_YR2_RT,FIRSTGEN_ENRL_4YR_TRANS_YR2_RT,FIRSTGEN_ENRL_2YR_TRANS_YR2_RT,FIRSTGEN_UNKN_ORIG_YR2_RT,FIRSTGEN_UNKN_4YR_TRANS_YR2_RT,FIRSTGEN_UNKN_2YR_TRANS_YR2_RT,NOT1STGEN_DEATH_YR2_RT,NOT1STGEN_COMP_ORIG_YR2_RT,NOT1STGEN_COMP_4YR_TRANS_YR2_RT,NOT1STGEN_COMP_2YR_TRANS_YR2_RT,NOT1STGEN_WDRAW_ORIG_YR2_RT,NOT1STGEN_WDRAW_4YR_TRANS_YR2_RT,NOT1STGEN_WDRAW_2YR_TRANS_YR2_RT,NOT1STGEN_ENRL_ORIG_YR2_RT,NOT1STGEN_ENRL_4YR_TRANS_YR2_RT,NOT1STGEN_ENRL_2YR_TRANS_YR2_RT,NOT1STGEN_UNKN_ORIG_YR2_RT,NOT1STGEN_UNKN_4YR_TRANS_YR2_RT,NOT1STGEN_UNKN_2YR_TRANS_YR2_RT,DEATH_YR3_RT,COMP_ORIG_YR3_RT,COMP_4YR_TRANS_YR3_RT,COMP_2YR_TRANS_YR3_RT,WDRAW_ORIG_YR3_RT,WDRAW_4YR_TRANS_YR3_RT,WDRAW_2YR_TRANS_YR3_RT,ENRL_ORIG_YR3_RT,ENRL_4YR_TRANS_YR3_RT,ENRL_2YR_TRANS_YR3_RT,UNKN_ORIG_YR3_RT,UNKN_4YR_TRANS_YR3_RT,UNKN_2YR_TRANS_YR3_RT,LO_INC_DEATH_YR3_RT,LO_INC_COMP_ORIG_YR3_RT,LO_INC_COMP_4YR_TRANS_YR3_RT,LO_INC_COMP_2YR_TRANS_YR3_RT,LO_INC_WDRAW_ORIG_YR3_RT,LO_INC_WDRAW_4YR_TRANS_YR3_RT,LO_INC_WDRAW_2YR_TRANS_YR3_RT,LO_INC_ENRL_ORIG_YR3_RT,LO_INC_ENRL_4YR_TRANS_YR3_RT,LO_INC_ENRL_2YR_TRANS_YR3_RT,LO_INC_UNKN_ORIG_YR3_RT,LO_INC_UNKN_4YR_TRANS_YR3_RT,LO_INC_UNKN_2YR_TRANS_YR3_RT,MD_INC_DEATH_YR3_RT,MD_INC_COMP_ORIG_YR3_RT,MD_INC_COMP_4YR_TRANS_YR3_RT,MD_INC_COMP_2YR_TRANS_YR3_RT,MD_INC_WDRAW_ORIG_YR3_RT,MD_INC_WDRAW_4YR_TRANS_YR3_RT,MD_INC_WDRAW_2YR_TRANS_YR3_RT,MD_INC_ENRL_ORIG_YR3_RT,MD_INC_ENRL_4YR_TRANS_YR3_RT,MD_INC_ENRL_2YR_TRANS_YR3_RT,MD_INC_UNKN_ORIG_YR3_RT,MD_INC_UNKN_4YR_TRANS_YR3_RT,MD_INC_UNKN_2YR_TRANS_YR3_RT,HI_INC_DEATH_YR3_RT,HI_INC_COMP_ORIG_YR3_RT,HI_INC_COMP_4YR_TRANS_YR3_RT,HI_INC_COMP_2YR_TRANS_YR3_RT,HI_INC_WDRAW_ORIG_YR3_RT,HI_INC_WDRAW_4YR_TRANS_YR3_RT,HI_INC_WDRAW_2YR_TRANS_YR3_RT,HI_INC_ENRL_ORIG_YR3_RT,HI_INC_ENRL_4YR_TRANS_YR3_RT,HI_INC_ENRL_2YR_TRANS_YR3_RT,HI_INC_UNKN_ORIG_YR3_RT,HI_INC_UNKN_4YR_TRANS_YR3_RT,HI_INC_UNKN_2YR_TRANS_YR3_RT,DEP_DEATH_YR3_RT,DEP_COMP_ORIG_YR3_RT,DEP_COMP_4YR_TRANS_YR3_RT,DEP_COMP_2YR_TRANS_YR3_RT,DEP_WDRAW_ORIG_YR3_RT,DEP_WDRAW_4YR_TRANS_YR3_RT,DEP_WDRAW_2YR_TRANS_YR3_RT,DEP_ENRL_ORIG_YR3_RT,DEP_ENRL_4YR_TRANS_YR3_RT,DEP_ENRL_2YR_TRANS_YR3_RT,DEP_UNKN_ORIG_YR3_RT,DEP_UNKN_4YR_TRANS_YR3_RT,DEP_UNKN_2YR_TRANS_YR3_RT,IND_DEATH_YR3_RT,IND_COMP_ORIG_YR3_RT,IND_COMP_4YR_TRANS_YR3_RT,IND_COMP_2YR_TRANS_YR3_RT,IND_WDRAW_ORIG_YR3_RT,IND_WDRAW_4YR_TRANS_YR3_RT,IND_WDRAW_2YR_TRANS_YR3_RT,IND_ENRL_ORIG_YR3_RT,IND_ENRL_4YR_TRANS_YR3_RT,IND_ENRL_2YR_TRANS_YR3_RT,IND_UNKN_ORIG_YR3_RT,IND_UNKN_4YR_TRANS_YR3_RT,IND_UNKN_2YR_TRANS_YR3_RT,FEMALE_DEATH_YR3_RT,FEMALE_COMP_ORIG_YR3_RT,FEMALE_COMP_4YR_TRANS_YR3_RT,FEMALE_COMP_2YR_TRANS_YR3_RT,FEMALE_WDRAW_ORIG_YR3_RT,FEMALE_WDRAW_4YR_TRANS_YR3_RT,FEMALE_WDRAW_2YR_TRANS_YR3_RT,FEMALE_ENRL_ORIG_YR3_RT,FEMALE_ENRL_4YR_TRANS_YR3_RT,FEMALE_ENRL_2YR_TRANS_YR3_RT,FEMALE_UNKN_ORIG_YR3_RT,FEMALE_UNKN_4YR_TRANS_YR3_RT,FEMALE_UNKN_2YR_TRANS_YR3_RT,MALE_DEATH_YR3_RT,MALE_COMP_ORIG_YR3_RT,MALE_COMP_4YR_TRANS_YR3_RT,MALE_COMP_2YR_TRANS_YR3_RT,MALE_WDRAW_ORIG_YR3_RT,MALE_WDRAW_4YR_TRANS_YR3_RT,MALE_WDRAW_2YR_TRANS_YR3_RT,MALE_ENRL_ORIG_YR3_RT,MALE_ENRL_4YR_TRANS_YR3_RT,MALE_ENRL_2YR_TRANS_YR3_RT,MALE_UNKN_ORIG_YR3_RT,MALE_UNKN_4YR_TRANS_YR3_RT,MALE_UNKN_2YR_TRANS_YR3_RT,PELL_DEATH_YR3_RT,PELL_COMP_ORIG_YR3_RT,PELL_COMP_4YR_TRANS_YR3_RT,PELL_COMP_2YR_TRANS_YR3_RT,PELL_WDRAW_ORIG_YR3_RT,PELL_WDRAW_4YR_TRANS_YR3_RT,PELL_WDRAW_2YR_TRANS_YR3_RT,PELL_ENRL_ORIG_YR3_RT,PELL_ENRL_4YR_TRANS_YR3_RT,PELL_ENRL_2YR_TRANS_YR3_RT,PELL_UNKN_ORIG_YR3_RT,PELL_UNKN_4YR_TRANS_YR3_RT,PELL_UNKN_2YR_TRANS_YR3_RT,NOPELL_DEATH_YR3_RT,NOPELL_COMP_ORIG_YR3_RT,NOPELL_COMP_4YR_TRANS_YR3_RT,NOPELL_COMP_2YR_TRANS_YR3_RT,NOPELL_WDRAW_ORIG_YR3_RT,NOPELL_WDRAW_4YR_TRANS_YR3_RT,NOPELL_WDRAW_2YR_TRANS_YR3_RT,NOPELL_ENRL_ORIG_YR3_RT,NOPELL_ENRL_4YR_TRANS_YR3_RT,NOPELL_ENRL_2YR_TRANS_YR3_RT,NOPELL_UNKN_ORIG_YR3_RT,NOPELL_UNKN_4YR_TRANS_YR3_RT,NOPELL_UNKN_2YR_TRANS_YR3_RT,LOAN_DEATH_YR3_RT,LOAN_COMP_ORIG_YR3_RT,LOAN_COMP_4YR_TRANS_YR3_RT,LOAN_COMP_2YR_TRANS_YR3_RT,LOAN_WDRAW_ORIG_YR3_RT,LOAN_WDRAW_4YR_TRANS_YR3_RT,LOAN_WDRAW_2YR_TRANS_YR3_RT,LOAN_ENRL_ORIG_YR3_RT,LOAN_ENRL_4YR_TRANS_YR3_RT,LOAN_ENRL_2YR_TRANS_YR3_RT,LOAN_UNKN_ORIG_YR3_RT,LOAN_UNKN_4YR_TRANS_YR3_RT,LOAN_UNKN_2YR_TRANS_YR3_RT,NOLOAN_DEATH_YR3_RT,NOLOAN_COMP_ORIG_YR3_RT,NOLOAN_COMP_4YR_TRANS_YR3_RT,NOLOAN_COMP_2YR_TRANS_YR3_RT,NOLOAN_WDRAW_ORIG_YR3_RT,NOLOAN_WDRAW_4YR_TRANS_YR3_RT,NOLOAN_WDRAW_2YR_TRANS_YR3_RT,NOLOAN_ENRL_ORIG_YR3_RT,NOLOAN_ENRL_4YR_TRANS_YR3_RT,NOLOAN_ENRL_2YR_TRANS_YR3_RT,NOLOAN_UNKN_ORIG_YR3_RT,NOLOAN_UNKN_4YR_TRANS_YR3_RT,NOLOAN_UNKN_2YR_TRANS_YR3_RT,FIRSTGEN_DEATH_YR3_RT,FIRSTGEN_COMP_ORIG_YR3_RT,FIRSTGEN_COMP_4YR_TRANS_YR3_RT,FIRSTGEN_COMP_2YR_TRANS_YR3_RT,FIRSTGEN_WDRAW_ORIG_YR3_RT,FIRSTGEN_WDRAW_4YR_TRANS_YR3_RT,FIRSTGEN_WDRAW_2YR_TRANS_YR3_RT,FIRSTGEN_ENRL_ORIG_YR3_RT,FIRSTGEN_ENRL_4YR_TRANS_YR3_RT,FIRSTGEN_ENRL_2YR_TRANS_YR3_RT,FIRSTGEN_UNKN_ORIG_YR3_RT,FIRSTGEN_UNKN_4YR_TRANS_YR3_RT,FIRSTGEN_UNKN_2YR_TRANS_YR3_RT,NOT1STGEN_DEATH_YR3_RT,NOT1STGEN_COMP_ORIG_YR3_RT,NOT1STGEN_COMP_4YR_TRANS_YR3_RT,NOT1STGEN_COMP_2YR_TRANS_YR3_RT,NOT1STGEN_WDRAW_ORIG_YR3_RT,NOT1STGEN_WDRAW_4YR_TRANS_YR3_RT,NOT1STGEN_WDRAW_2YR_TRANS_YR3_RT,NOT1STGEN_ENRL_ORIG_YR3_RT,NOT1STGEN_ENRL_4YR_TRANS_YR3_RT,NOT1STGEN_ENRL_2YR_TRANS_YR3_RT,NOT1STGEN_UNKN_ORIG_YR3_RT,NOT1STGEN_UNKN_4YR_TRANS_YR3_RT,NOT1STGEN_UNKN_2YR_TRANS_YR3_RT,DEATH_YR4_RT,COMP_ORIG_YR4_RT,COMP_4YR_TRANS_YR4_RT,COMP_2YR_TRANS_YR4_RT,WDRAW_ORIG_YR4_RT,WDRAW_4YR_TRANS_YR4_RT,WDRAW_2YR_TRANS_YR4_RT,ENRL_ORIG_YR4_RT,ENRL_4YR_TRANS_YR4_RT,ENRL_2YR_TRANS_YR4_RT,UNKN_ORIG_YR4_RT,UNKN_4YR_TRANS_YR4_RT,UNKN_2YR_TRANS_YR4_RT,LO_INC_DEATH_YR4_RT,LO_INC_COMP_ORIG_YR4_RT,LO_INC_COMP_4YR_TRANS_YR4_RT,LO_INC_COMP_2YR_TRANS_YR4_RT,LO_INC_WDRAW_ORIG_YR4_RT,LO_INC_WDRAW_4YR_TRANS_YR4_RT,LO_INC_WDRAW_2YR_TRANS_YR4_RT,LO_INC_ENRL_ORIG_YR4_RT,LO_INC_ENRL_4YR_TRANS_YR4_RT,LO_INC_ENRL_2YR_TRANS_YR4_RT,LO_INC_UNKN_ORIG_YR4_RT,LO_INC_UNKN_4YR_TRANS_YR4_RT,LO_INC_UNKN_2YR_TRANS_YR4_RT,MD_INC_DEATH_YR4_RT,MD_INC_COMP_ORIG_YR4_RT,MD_INC_COMP_4YR_TRANS_YR4_RT,MD_INC_COMP_2YR_TRANS_YR4_RT,MD_INC_WDRAW_ORIG_YR4_RT,MD_INC_WDRAW_4YR_TRANS_YR4_RT,MD_INC_WDRAW_2YR_TRANS_YR4_RT,MD_INC_ENRL_ORIG_YR4_RT,MD_INC_ENRL_4YR_TRANS_YR4_RT,MD_INC_ENRL_2YR_TRANS_YR4_RT,MD_INC_UNKN_ORIG_YR4_RT,MD_INC_UNKN_4YR_TRANS_YR4_RT,MD_INC_UNKN_2YR_TRANS_YR4_RT,HI_INC_DEATH_YR4_RT,HI_INC_COMP_ORIG_YR4_RT,HI_INC_COMP_4YR_TRANS_YR4_RT,HI_INC_COMP_2YR_TRANS_YR4_RT,HI_INC_WDRAW_ORIG_YR4_RT,HI_INC_WDRAW_4YR_TRANS_YR4_RT,HI_INC_WDRAW_2YR_TRANS_YR4_RT,HI_INC_ENRL_ORIG_YR4_RT,HI_INC_ENRL_4YR_TRANS_YR4_RT,HI_INC_ENRL_2YR_TRANS_YR4_RT,HI_INC_UNKN_ORIG_YR4_RT,HI_INC_UNKN_4YR_TRANS_YR4_RT,HI_INC_UNKN_2YR_TRANS_YR4_RT,DEP_DEATH_YR4_RT,DEP_COMP_ORIG_YR4_RT,DEP_COMP_4YR_TRANS_YR4_RT,DEP_COMP_2YR_TRANS_YR4_RT,DEP_WDRAW_ORIG_YR4_RT,DEP_WDRAW_4YR_TRANS_YR4_RT,DEP_WDRAW_2YR_TRANS_YR4_RT,DEP_ENRL_ORIG_YR4_RT,DEP_ENRL_4YR_TRANS_YR4_RT,DEP_ENRL_2YR_TRANS_YR4_RT,DEP_UNKN_ORIG_YR4_RT,DEP_UNKN_4YR_TRANS_YR4_RT,DEP_UNKN_2YR_TRANS_YR4_RT,IND_DEATH_YR4_RT,IND_COMP_ORIG_YR4_RT,IND_COMP_4YR_TRANS_YR4_RT,IND_COMP_2YR_TRANS_YR4_RT,IND_WDRAW_ORIG_YR4_RT,IND_WDRAW_4YR_TRANS_YR4_RT,IND_WDRAW_2YR_TRANS_YR4_RT,IND_ENRL_ORIG_YR4_RT,IND_ENRL_4YR_TRANS_YR4_RT,IND_ENRL_2YR_TRANS_YR4_RT,IND_UNKN_ORIG_YR4_RT,IND_UNKN_4YR_TRANS_YR4_RT,IND_UNKN_2YR_TRANS_YR4_RT,FEMALE_DEATH_YR4_RT,FEMALE_COMP_ORIG_YR4_RT,FEMALE_COMP_4YR_TRANS_YR4_RT,FEMALE_COMP_2YR_TRANS_YR4_RT,FEMALE_WDRAW_ORIG_YR4_RT,FEMALE_WDRAW_4YR_TRANS_YR4_RT,FEMALE_WDRAW_2YR_TRANS_YR4_RT,FEMALE_ENRL_ORIG_YR4_RT,FEMALE_ENRL_4YR_TRANS_YR4_RT,FEMALE_ENRL_2YR_TRANS_YR4_RT,FEMALE_UNKN_ORIG_YR4_RT,FEMALE_UNKN_4YR_TRANS_YR4_RT,FEMALE_UNKN_2YR_TRANS_YR4_RT,MALE_DEATH_YR4_RT,MALE_COMP_ORIG_YR4_RT,MALE_COMP_4YR_TRANS_YR4_RT,MALE_COMP_2YR_TRANS_YR4_RT,MALE_WDRAW_ORIG_YR4_RT,MALE_WDRAW_4YR_TRANS_YR4_RT,MALE_WDRAW_2YR_TRANS_YR4_RT,MALE_ENRL_ORIG_YR4_RT,MALE_ENRL_4YR_TRANS_YR4_RT,MALE_ENRL_2YR_TRANS_YR4_RT,MALE_UNKN_ORIG_YR4_RT,MALE_UNKN_4YR_TRANS_YR4_RT,MALE_UNKN_2YR_TRANS_YR4_RT,PELL_DEATH_YR4_RT,PELL_COMP_ORIG_YR4_RT,PELL_COMP_4YR_TRANS_YR4_RT,PELL_COMP_2YR_TRANS_YR4_RT,PELL_WDRAW_ORIG_YR4_RT,PELL_WDRAW_4YR_TRANS_YR4_RT,PELL_WDRAW_2YR_TRANS_YR4_RT,PELL_ENRL_ORIG_YR4_RT,PELL_ENRL_4YR_TRANS_YR4_RT,PELL_ENRL_2YR_TRANS_YR4_RT,PELL_UNKN_ORIG_YR4_RT,PELL_UNKN_4YR_TRANS_YR4_RT,PELL_UNKN_2YR_TRANS_YR4_RT,NOPELL_DEATH_YR4_RT,NOPELL_COMP_ORIG_YR4_RT,NOPELL_COMP_4YR_TRANS_YR4_RT,NOPELL_COMP_2YR_TRANS_YR4_RT,NOPELL_WDRAW_ORIG_YR4_RT,NOPELL_WDRAW_4YR_TRANS_YR4_RT,NOPELL_WDRAW_2YR_TRANS_YR4_RT,NOPELL_ENRL_ORIG_YR4_RT,NOPELL_ENRL_4YR_TRANS_YR4_RT,NOPELL_ENRL_2YR_TRANS_YR4_RT,NOPELL_UNKN_ORIG_YR4_RT,NOPELL_UNKN_4YR_TRANS_YR4_RT,NOPELL_UNKN_2YR_TRANS_YR4_RT,LOAN_DEATH_YR4_RT,LOAN_COMP_ORIG_YR4_RT,LOAN_COMP_4YR_TRANS_YR4_RT,LOAN_COMP_2YR_TRANS_YR4_RT,LOAN_WDRAW_ORIG_YR4_RT,LOAN_WDRAW_4YR_TRANS_YR4_RT,LOAN_WDRAW_2YR_TRANS_YR4_RT,LOAN_ENRL_ORIG_YR4_RT,LOAN_ENRL_4YR_TRANS_YR4_RT,LOAN_ENRL_2YR_TRANS_YR4_RT,LOAN_UNKN_ORIG_YR4_RT,LOAN_UNKN_4YR_TRANS_YR4_RT,LOAN_UNKN_2YR_TRANS_YR4_RT,NOLOAN_DEATH_YR4_RT,NOLOAN_COMP_ORIG_YR4_RT,NOLOAN_COMP_4YR_TRANS_YR4_RT,NOLOAN_COMP_2YR_TRANS_YR4_RT,NOLOAN_WDRAW_ORIG_YR4_RT,NOLOAN_WDRAW_4YR_TRANS_YR4_RT,NOLOAN_WDRAW_2YR_TRANS_YR4_RT,NOLOAN_ENRL_ORIG_YR4_RT,NOLOAN_ENRL_4YR_TRANS_YR4_RT,NOLOAN_ENRL_2YR_TRANS_YR4_RT,NOLOAN_UNKN_ORIG_YR4_RT,NOLOAN_UNKN_4YR_TRANS_YR4_RT,NOLOAN_UNKN_2YR_TRANS_YR4_RT,FIRSTGEN_DEATH_YR4_RT,FIRSTGEN_COMP_ORIG_YR4_RT,FIRSTGEN_COMP_4YR_TRANS_YR4_RT,FIRSTGEN_COMP_2YR_TRANS_YR4_RT,FIRSTGEN_WDRAW_ORIG_YR4_RT,FIRSTGEN_WDRAW_4YR_TRANS_YR4_RT,FIRSTGEN_WDRAW_2YR_TRANS_YR4_RT,FIRSTGEN_ENRL_ORIG_YR4_RT,FIRSTGEN_ENRL_4YR_TRANS_YR4_RT,FIRSTGEN_ENRL_2YR_TRANS_YR4_RT,FIRSTGEN_UNKN_ORIG_YR4_RT,FIRSTGEN_UNKN_4YR_TRANS_YR4_RT,FIRSTGEN_UNKN_2YR_TRANS_YR4_RT,NOT1STGEN_DEATH_YR4_RT,NOT1STGEN_COMP_ORIG_YR4_RT,NOT1STGEN_COMP_4YR_TRANS_YR4_RT,NOT1STGEN_COMP_2YR_TRANS_YR4_RT,NOT1STGEN_WDRAW_ORIG_YR4_RT,NOT1STGEN_WDRAW_4YR_TRANS_YR4_RT,NOT1STGEN_WDRAW_2YR_TRANS_YR4_RT,NOT1STGEN_ENRL_ORIG_YR4_RT,NOT1STGEN_ENRL_4YR_TRANS_YR4_RT,NOT1STGEN_ENRL_2YR_TRANS_YR4_RT,NOT1STGEN_UNKN_ORIG_YR4_RT,NOT1STGEN_UNKN_4YR_TRANS_YR4_RT,NOT1STGEN_UNKN_2YR_TRANS_YR4_RT,DEATH_YR6_RT,COMP_ORIG_YR6_RT,COMP_4YR_TRANS_YR6_RT,COMP_2YR_TRANS_YR6_RT,WDRAW_ORIG_YR6_RT,WDRAW_4YR_TRANS_YR6_RT,WDRAW_2YR_TRANS_YR6_RT,ENRL_ORIG_YR6_RT,ENRL_4YR_TRANS_YR6_RT,ENRL_2YR_TRANS_YR6_RT,UNKN_ORIG_YR6_RT,UNKN_4YR_TRANS_YR6_RT,UNKN_2YR_TRANS_YR6_RT,LO_INC_DEATH_YR6_RT,LO_INC_COMP_ORIG_YR6_RT,LO_INC_COMP_4YR_TRANS_YR6_RT,LO_INC_COMP_2YR_TRANS_YR6_RT,LO_INC_WDRAW_ORIG_YR6_RT,LO_INC_WDRAW_4YR_TRANS_YR6_RT,LO_INC_WDRAW_2YR_TRANS_YR6_RT,LO_INC_ENRL_ORIG_YR6_RT,LO_INC_ENRL_4YR_TRANS_YR6_RT,LO_INC_ENRL_2YR_TRANS_YR6_RT,LO_INC_UNKN_ORIG_YR6_RT,LO_INC_UNKN_4YR_TRANS_YR6_RT,LO_INC_UNKN_2YR_TRANS_YR6_RT,MD_INC_DEATH_YR6_RT,MD_INC_COMP_ORIG_YR6_RT,MD_INC_COMP_4YR_TRANS_YR6_RT,MD_INC_COMP_2YR_TRANS_YR6_RT,MD_INC_WDRAW_ORIG_YR6_RT,MD_INC_WDRAW_4YR_TRANS_YR6_RT,MD_INC_WDRAW_2YR_TRANS_YR6_RT,MD_INC_ENRL_ORIG_YR6_RT,MD_INC_ENRL_4YR_TRANS_YR6_RT,MD_INC_ENRL_2YR_TRANS_YR6_RT,MD_INC_UNKN_ORIG_YR6_RT,MD_INC_UNKN_4YR_TRANS_YR6_RT,MD_INC_UNKN_2YR_TRANS_YR6_RT,HI_INC_DEATH_YR6_RT,HI_INC_COMP_ORIG_YR6_RT,HI_INC_COMP_4YR_TRANS_YR6_RT,HI_INC_COMP_2YR_TRANS_YR6_RT,HI_INC_WDRAW_ORIG_YR6_RT,HI_INC_WDRAW_4YR_TRANS_YR6_RT,HI_INC_WDRAW_2YR_TRANS_YR6_RT,HI_INC_ENRL_ORIG_YR6_RT,HI_INC_ENRL_4YR_TRANS_YR6_RT,HI_INC_ENRL_2YR_TRANS_YR6_RT,HI_INC_UNKN_ORIG_YR6_RT,HI_INC_UNKN_4YR_TRANS_YR6_RT,HI_INC_UNKN_2YR_TRANS_YR6_RT,DEP_DEATH_YR6_RT,DEP_COMP_ORIG_YR6_RT,DEP_COMP_4YR_TRANS_YR6_RT,DEP_COMP_2YR_TRANS_YR6_RT,DEP_WDRAW_ORIG_YR6_RT,DEP_WDRAW_4YR_TRANS_YR6_RT,DEP_WDRAW_2YR_TRANS_YR6_RT,DEP_ENRL_ORIG_YR6_RT,DEP_ENRL_4YR_TRANS_YR6_RT,DEP_ENRL_2YR_TRANS_YR6_RT,DEP_UNKN_ORIG_YR6_RT,DEP_UNKN_4YR_TRANS_YR6_RT,DEP_UNKN_2YR_TRANS_YR6_RT,IND_DEATH_YR6_RT,IND_COMP_ORIG_YR6_RT,IND_COMP_4YR_TRANS_YR6_RT,IND_COMP_2YR_TRANS_YR6_RT,IND_WDRAW_ORIG_YR6_RT,IND_WDRAW_4YR_TRANS_YR6_RT,IND_WDRAW_2YR_TRANS_YR6_RT,IND_ENRL_ORIG_YR6_RT,IND_ENRL_4YR_TRANS_YR6_RT,IND_ENRL_2YR_TRANS_YR6_RT,IND_UNKN_ORIG_YR6_RT,IND_UNKN_4YR_TRANS_YR6_RT,IND_UNKN_2YR_TRANS_YR6_RT,FEMALE_DEATH_YR6_RT,FEMALE_COMP_ORIG_YR6_RT,FEMALE_COMP_4YR_TRANS_YR6_RT,FEMALE_COMP_2YR_TRANS_YR6_RT,FEMALE_WDRAW_ORIG_YR6_RT,FEMALE_WDRAW_4YR_TRANS_YR6_RT,FEMALE_WDRAW_2YR_TRANS_YR6_RT,FEMALE_ENRL_ORIG_YR6_RT,FEMALE_ENRL_4YR_TRANS_YR6_RT,FEMALE_ENRL_2YR_TRANS_YR6_RT,FEMALE_UNKN_ORIG_YR6_RT,FEMALE_UNKN_4YR_TRANS_YR6_RT,FEMALE_UNKN_2YR_TRANS_YR6_RT,MALE_DEATH_YR6_RT,MALE_COMP_ORIG_YR6_RT,MALE_COMP_4YR_TRANS_YR6_RT,MALE_COMP_2YR_TRANS_YR6_RT,MALE_WDRAW_ORIG_YR6_RT,MALE_WDRAW_4YR_TRANS_YR6_RT,MALE_WDRAW_2YR_TRANS_YR6_RT,MALE_ENRL_ORIG_YR6_RT,MALE_ENRL_4YR_TRANS_YR6_RT,MALE_ENRL_2YR_TRANS_YR6_RT,MALE_UNKN_ORIG_YR6_RT,MALE_UNKN_4YR_TRANS_YR6_RT,MALE_UNKN_2YR_TRANS_YR6_RT,PELL_DEATH_YR6_RT,PELL_COMP_ORIG_YR6_RT,PELL_COMP_4YR_TRANS_YR6_RT,PELL_COMP_2YR_TRANS_YR6_RT,PELL_WDRAW_ORIG_YR6_RT,PELL_WDRAW_4YR_TRANS_YR6_RT,PELL_WDRAW_2YR_TRANS_YR6_RT,PELL_ENRL_ORIG_YR6_RT,PELL_ENRL_4YR_TRANS_YR6_RT,PELL_ENRL_2YR_TRANS_YR6_RT,PELL_UNKN_ORIG_YR6_RT,PELL_UNKN_4YR_TRANS_YR6_RT,PELL_UNKN_2YR_TRANS_YR6_RT,NOPELL_DEATH_YR6_RT,NOPELL_COMP_ORIG_YR6_RT,NOPELL_COMP_4YR_TRANS_YR6_RT,NOPELL_COMP_2YR_TRANS_YR6_RT,NOPELL_WDRAW_ORIG_YR6_RT,NOPELL_WDRAW_4YR_TRANS_YR6_RT,NOPELL_WDRAW_2YR_TRANS_YR6_RT,NOPELL_ENRL_ORIG_YR6_RT,NOPELL_ENRL_4YR_TRANS_YR6_RT,NOPELL_ENRL_2YR_TRANS_YR6_RT,NOPELL_UNKN_ORIG_YR6_RT,NOPELL_UNKN_4YR_TRANS_YR6_RT,NOPELL_UNKN_2YR_TRANS_YR6_RT,LOAN_DEATH_YR6_RT,LOAN_COMP_ORIG_YR6_RT,LOAN_COMP_4YR_TRANS_YR6_RT,LOAN_COMP_2YR_TRANS_YR6_RT,LOAN_WDRAW_ORIG_YR6_RT,LOAN_WDRAW_4YR_TRANS_YR6_RT,LOAN_WDRAW_2YR_TRANS_YR6_RT,LOAN_ENRL_ORIG_YR6_RT,LOAN_ENRL_4YR_TRANS_YR6_RT,LOAN_ENRL_2YR_TRANS_YR6_RT,LOAN_UNKN_ORIG_YR6_RT,LOAN_UNKN_4YR_TRANS_YR6_RT,LOAN_UNKN_2YR_TRANS_YR6_RT,NOLOAN_DEATH_YR6_RT,NOLOAN_COMP_ORIG_YR6_RT,NOLOAN_COMP_4YR_TRANS_YR6_RT,NOLOAN_COMP_2YR_TRANS_YR6_RT,NOLOAN_WDRAW_ORIG_YR6_RT,NOLOAN_WDRAW_4YR_TRANS_YR6_RT,NOLOAN_WDRAW_2YR_TRANS_YR6_RT,NOLOAN_ENRL_ORIG_YR6_RT,NOLOAN_ENRL_4YR_TRANS_YR6_RT,NOLOAN_ENRL_2YR_TRANS_YR6_RT,NOLOAN_UNKN_ORIG_YR6_RT,NOLOAN_UNKN_4YR_TRANS_YR6_RT,NOLOAN_UNKN_2YR_TRANS_YR6_RT,FIRSTGEN_DEATH_YR6_RT,FIRSTGEN_COMP_ORIG_YR6_RT,FIRSTGEN_COMP_4YR_TRANS_YR6_RT,FIRSTGEN_COMP_2YR_TRANS_YR6_RT,FIRSTGEN_WDRAW_ORIG_YR6_RT,FIRSTGEN_WDRAW_4YR_TRANS_YR6_RT,FIRSTGEN_WDRAW_2YR_TRANS_YR6_RT,FIRSTGEN_ENRL_ORIG_YR6_RT,FIRSTGEN_ENRL_4YR_TRANS_YR6_RT,FIRSTGEN_ENRL_2YR_TRANS_YR6_RT,FIRSTGEN_UNKN_ORIG_YR6_RT,FIRSTGEN_UNKN_4YR_TRANS_YR6_RT,FIRSTGEN_UNKN_2YR_TRANS_YR6_RT,NOT1STGEN_DEATH_YR6_RT,NOT1STGEN_COMP_ORIG_YR6_RT,NOT1STGEN_COMP_4YR_TRANS_YR6_RT,NOT1STGEN_COMP_2YR_TRANS_YR6_RT,NOT1STGEN_WDRAW_ORIG_YR6_RT,NOT1STGEN_WDRAW_4YR_TRANS_YR6_RT,NOT1STGEN_WDRAW_2YR_TRANS_YR6_RT,NOT1STGEN_ENRL_ORIG_YR6_RT,NOT1STGEN_ENRL_4YR_TRANS_YR6_RT,NOT1STGEN_ENRL_2YR_TRANS_YR6_RT,NOT1STGEN_UNKN_ORIG_YR6_RT,NOT1STGEN_UNKN_4YR_TRANS_YR6_RT,NOT1STGEN_UNKN_2YR_TRANS_YR6_RT,DEATH_YR8_RT,COMP_ORIG_YR8_RT,COMP_4YR_TRANS_YR8_RT,COMP_2YR_TRANS_YR8_RT,WDRAW_ORIG_YR8_RT,WDRAW_4YR_TRANS_YR8_RT,WDRAW_2YR_TRANS_YR8_RT,ENRL_ORIG_YR8_RT,ENRL_4YR_TRANS_YR8_RT,ENRL_2YR_TRANS_YR8_RT,UNKN_ORIG_YR8_RT,UNKN_4YR_TRANS_YR8_RT,UNKN_2YR_TRANS_YR8_RT,LO_INC_DEATH_YR8_RT,LO_INC_COMP_ORIG_YR8_RT,LO_INC_COMP_4YR_TRANS_YR8_RT,LO_INC_COMP_2YR_TRANS_YR8_RT,LO_INC_WDRAW_ORIG_YR8_RT,LO_INC_WDRAW_4YR_TRANS_YR8_RT,LO_INC_WDRAW_2YR_TRANS_YR8_RT,LO_INC_ENRL_ORIG_YR8_RT,LO_INC_ENRL_4YR_TRANS_YR8_RT,LO_INC_ENRL_2YR_TRANS_YR8_RT,LO_INC_UNKN_ORIG_YR8_RT,LO_INC_UNKN_4YR_TRANS_YR8_RT,LO_INC_UNKN_2YR_TRANS_YR8_RT,MD_INC_DEATH_YR8_RT,MD_INC_COMP_ORIG_YR8_RT,MD_INC_COMP_4YR_TRANS_YR8_RT,MD_INC_COMP_2YR_TRANS_YR8_RT,MD_INC_WDRAW_ORIG_YR8_RT,MD_INC_WDRAW_4YR_TRANS_YR8_RT,MD_INC_WDRAW_2YR_TRANS_YR8_RT,MD_INC_ENRL_ORIG_YR8_RT,MD_INC_ENRL_4YR_TRANS_YR8_RT,MD_INC_ENRL_2YR_TRANS_YR8_RT,MD_INC_UNKN_ORIG_YR8_RT,MD_INC_UNKN_4YR_TRANS_YR8_RT,MD_INC_UNKN_2YR_TRANS_YR8_RT,HI_INC_DEATH_YR8_RT,HI_INC_COMP_ORIG_YR8_RT,HI_INC_COMP_4YR_TRANS_YR8_RT,HI_INC_COMP_2YR_TRANS_YR8_RT,HI_INC_WDRAW_ORIG_YR8_RT,HI_INC_WDRAW_4YR_TRANS_YR8_RT,HI_INC_WDRAW_2YR_TRANS_YR8_RT,HI_INC_ENRL_ORIG_YR8_RT,HI_INC_ENRL_4YR_TRANS_YR8_RT,HI_INC_ENRL_2YR_TRANS_YR8_RT,HI_INC_UNKN_ORIG_YR8_RT,HI_INC_UNKN_4YR_TRANS_YR8_RT,HI_INC_UNKN_2YR_TRANS_YR8_RT,DEP_DEATH_YR8_RT,DEP_COMP_ORIG_YR8_RT,DEP_COMP_4YR_TRANS_YR8_RT,DEP_COMP_2YR_TRANS_YR8_RT,DEP_WDRAW_ORIG_YR8_RT,DEP_WDRAW_4YR_TRANS_YR8_RT,DEP_WDRAW_2YR_TRANS_YR8_RT,DEP_ENRL_ORIG_YR8_RT,DEP_ENRL_4YR_TRANS_YR8_RT,DEP_ENRL_2YR_TRANS_YR8_RT,DEP_UNKN_ORIG_YR8_RT,DEP_UNKN_4YR_TRANS_YR8_RT,DEP_UNKN_2YR_TRANS_YR8_RT,IND_DEATH_YR8_RT,IND_COMP_ORIG_YR8_RT,IND_COMP_4YR_TRANS_YR8_RT,IND_COMP_2YR_TRANS_YR8_RT,IND_WDRAW_ORIG_YR8_RT,IND_WDRAW_4YR_TRANS_YR8_RT,IND_WDRAW_2YR_TRANS_YR8_RT,IND_ENRL_ORIG_YR8_RT,IND_ENRL_4YR_TRANS_YR8_RT,IND_ENRL_2YR_TRANS_YR8_RT,IND_UNKN_ORIG_YR8_RT,IND_UNKN_4YR_TRANS_YR8_RT,IND_UNKN_2YR_TRANS_YR8_RT,FEMALE_DEATH_YR8_RT,FEMALE_COMP_ORIG_YR8_RT,FEMALE_COMP_4YR_TRANS_YR8_RT,FEMALE_COMP_2YR_TRANS_YR8_RT,FEMALE_WDRAW_ORIG_YR8_RT,FEMALE_WDRAW_4YR_TRANS_YR8_RT,FEMALE_WDRAW_2YR_TRANS_YR8_RT,FEMALE_ENRL_ORIG_YR8_RT,FEMALE_ENRL_4YR_TRANS_YR8_RT,FEMALE_ENRL_2YR_TRANS_YR8_RT,FEMALE_UNKN_ORIG_YR8_RT,FEMALE_UNKN_4YR_TRANS_YR8_RT,FEMALE_UNKN_2YR_TRANS_YR8_RT,MALE_DEATH_YR8_RT,MALE_COMP_ORIG_YR8_RT,MALE_COMP_4YR_TRANS_YR8_RT,MALE_COMP_2YR_TRANS_YR8_RT,MALE_WDRAW_ORIG_YR8_RT,MALE_WDRAW_4YR_TRANS_YR8_RT,MALE_WDRAW_2YR_TRANS_YR8_RT,MALE_ENRL_ORIG_YR8_RT,MALE_ENRL_4YR_TRANS_YR8_RT,MALE_ENRL_2YR_TRANS_YR8_RT,MALE_UNKN_ORIG_YR8_RT,MALE_UNKN_4YR_TRANS_YR8_RT,MALE_UNKN_2YR_TRANS_YR8_RT,PELL_DEATH_YR8_RT,PELL_COMP_ORIG_YR8_RT,PELL_COMP_4YR_TRANS_YR8_RT,PELL_COMP_2YR_TRANS_YR8_RT,PELL_WDRAW_ORIG_YR8_RT,PELL_WDRAW_4YR_TRANS_YR8_RT,PELL_WDRAW_2YR_TRANS_YR8_RT,PELL_ENRL_ORIG_YR8_RT,PELL_ENRL_4YR_TRANS_YR8_RT,PELL_ENRL_2YR_TRANS_YR8_RT,PELL_UNKN_ORIG_YR8_RT,PELL_UNKN_4YR_TRANS_YR8_RT,PELL_UNKN_2YR_TRANS_YR8_RT,NOPELL_DEATH_YR8_RT,NOPELL_COMP_ORIG_YR8_RT,NOPELL_COMP_4YR_TRANS_YR8_RT,NOPELL_COMP_2YR_TRANS_YR8_RT,NOPELL_WDRAW_ORIG_YR8_RT,NOPELL_WDRAW_4YR_TRANS_YR8_RT,NOPELL_WDRAW_2YR_TRANS_YR8_RT,NOPELL_ENRL_ORIG_YR8_RT,NOPELL_ENRL_4YR_TRANS_YR8_RT,NOPELL_ENRL_2YR_TRANS_YR8_RT,NOPELL_UNKN_ORIG_YR8_RT,NOPELL_UNKN_4YR_TRANS_YR8_RT,NOPELL_UNKN_2YR_TRANS_YR8_RT,LOAN_DEATH_YR8_RT,LOAN_COMP_ORIG_YR8_RT,LOAN_COMP_4YR_TRANS_YR8_RT,LOAN_COMP_2YR_TRANS_YR8_RT,LOAN_WDRAW_ORIG_YR8_RT,LOAN_WDRAW_4YR_TRANS_YR8_RT,LOAN_WDRAW_2YR_TRANS_YR8_RT,LOAN_ENRL_ORIG_YR8_RT,LOAN_ENRL_4YR_TRANS_YR8_RT,LOAN_ENRL_2YR_TRANS_YR8_RT,LOAN_UNKN_ORIG_YR8_RT,LOAN_UNKN_4YR_TRANS_YR8_RT,LOAN_UNKN_2YR_TRANS_YR8_RT,NOLOAN_DEATH_YR8_RT,NOLOAN_COMP_ORIG_YR8_RT,NOLOAN_COMP_4YR_TRANS_YR8_RT,NOLOAN_COMP_2YR_TRANS_YR8_RT,NOLOAN_WDRAW_ORIG_YR8_RT,NOLOAN_WDRAW_4YR_TRANS_YR8_RT,NOLOAN_WDRAW_2YR_TRANS_YR8_RT,NOLOAN_ENRL_ORIG_YR8_RT,NOLOAN_ENRL_4YR_TRANS_YR8_RT,NOLOAN_ENRL_2YR_TRANS_YR8_RT,NOLOAN_UNKN_ORIG_YR8_RT,NOLOAN_UNKN_4YR_TRANS_YR8_RT,NOLOAN_UNKN_2YR_TRANS_YR8_RT,FIRSTGEN_DEATH_YR8_RT,FIRSTGEN_COMP_ORIG_YR8_RT,FIRSTGEN_COMP_4YR_TRANS_YR8_RT,FIRSTGEN_COMP_2YR_TRANS_YR8_RT,FIRSTGEN_WDRAW_ORIG_YR8_RT,FIRSTGEN_WDRAW_4YR_TRANS_YR8_RT,FIRSTGEN_WDRAW_2YR_TRANS_YR8_RT,FIRSTGEN_ENRL_ORIG_YR8_RT,FIRSTGEN_ENRL_4YR_TRANS_YR8_RT,FIRSTGEN_ENRL_2YR_TRANS_YR8_RT,FIRSTGEN_UNKN_ORIG_YR8_RT,FIRSTGEN_UNKN_4YR_TRANS_YR8_RT,FIRSTGEN_UNKN_2YR_TRANS_YR8_RT,NOT1STGEN_DEATH_YR8_RT,NOT1STGEN_COMP_ORIG_YR8_RT,NOT1STGEN_COMP_4YR_TRANS_YR8_RT,NOT1STGEN_COMP_2YR_TRANS_YR8_RT,NOT1STGEN_WDRAW_ORIG_YR8_RT,NOT1STGEN_WDRAW_4YR_TRANS_YR8_RT,NOT1STGEN_WDRAW_2YR_TRANS_YR8_RT,NOT1STGEN_ENRL_ORIG_YR8_RT,NOT1STGEN_ENRL_4YR_TRANS_YR8_RT,NOT1STGEN_ENRL_2YR_TRANS_YR8_RT,NOT1STGEN_UNKN_ORIG_YR8_RT,NOT1STGEN_UNKN_4YR_TRANS_YR8_RT,NOT1STGEN_UNKN_2YR_TRANS_YR8_RT,RPY_1YR_RT,COMPL_RPY_1YR_RT,NONCOM_RPY_1YR_RT,LO_INC_RPY_1YR_RT,MD_INC_RPY_1YR_RT,HI_INC_RPY_1YR_RT,DEP_RPY_1YR_RT,IND_RPY_1YR_RT,PELL_RPY_1YR_RT,NOPELL_RPY_1YR_RT,FEMALE_RPY_1YR_RT,MALE_RPY_1YR_RT,FIRSTGEN_RPY_1YR_RT,NOTFIRSTGEN_RPY_1YR_RT,RPY_3YR_RT,COMPL_RPY_3YR_RT,NONCOM_RPY_3YR_RT,LO_INC_RPY_3YR_RT,MD_INC_RPY_3YR_RT,HI_INC_RPY_3YR_RT,DEP_RPY_3YR_RT,IND_RPY_3YR_RT,PELL_RPY_3YR_RT,NOPELL_RPY_3YR_RT,FEMALE_RPY_3YR_RT,MALE_RPY_3YR_RT,FIRSTGEN_RPY_3YR_RT,NOTFIRSTGEN_RPY_3YR_RT,RPY_5YR_RT,COMPL_RPY_5YR_RT,NONCOM_RPY_5YR_RT,LO_INC_RPY_5YR_RT,MD_INC_RPY_5YR_RT,HI_INC_RPY_5YR_RT,DEP_RPY_5YR_RT,IND_RPY_5YR_RT,PELL_RPY_5YR_RT,NOPELL_RPY_5YR_RT,FEMALE_RPY_5YR_RT,MALE_RPY_5YR_RT,FIRSTGEN_RPY_5YR_RT,NOTFIRSTGEN_RPY_5YR_RT,RPY_7YR_RT,COMPL_RPY_7YR_RT,NONCOM_RPY_7YR_RT,LO_INC_RPY_7YR_RT,MD_INC_RPY_7YR_RT,HI_INC_RPY_7YR_RT,DEP_RPY_7YR_RT,IND_RPY_7YR_RT,PELL_RPY_7YR_RT,NOPELL_RPY_7YR_RT,FEMALE_RPY_7YR_RT,MALE_RPY_7YR_RT,FIRSTGEN_RPY_7YR_RT,NOTFIRSTGEN_RPY_7YR_RT,INC_PCT_LO,DEP_STAT_PCT_IND,DEP_INC_PCT_LO,IND_INC_PCT_LO,PAR_ED_PCT_1STGEN,INC_PCT_M1,INC_PCT_M2,INC_PCT_H1,INC_PCT_H2,DEP_INC_PCT_M1,DEP_INC_PCT_M2,DEP_INC_PCT_H1,DEP_INC_PCT_H2,IND_INC_PCT_M1,IND_INC_PCT_M2,IND_INC_PCT_H1,IND_INC_PCT_H2,PAR_ED_PCT_MS,PAR_ED_PCT_HS,PAR_ED_PCT_PS,APPL_SCH_PCT_GE2,APPL_SCH_PCT_GE3,APPL_SCH_PCT_GE4,APPL_SCH_PCT_GE5,DEP_INC_AVG,IND_INC_AVG,OVERALL_YR2_N,LO_INC_YR2_N,MD_INC_YR2_N,HI_INC_YR2_N,DEP_YR2_N,IND_YR2_N,FEMALE_YR2_N,MALE_YR2_N,PELL_YR2_N,NOPELL_YR2_N,LOAN_YR2_N,NOLOAN_YR2_N,FIRSTGEN_YR2_N,NOT1STGEN_YR2_N,OVERALL_YR3_N,LO_INC_YR3_N,MD_INC_YR3_N,HI_INC_YR3_N,DEP_YR3_N,IND_YR3_N,FEMALE_YR3_N,MALE_YR3_N,PELL_YR3_N,NOPELL_YR3_N,LOAN_YR3_N,NOLOAN_YR3_N,FIRSTGEN_YR3_N,NOT1STGEN_YR3_N,OVERALL_YR4_N,LO_INC_YR4_N,MD_INC_YR4_N,HI_INC_YR4_N,DEP_YR4_N,IND_YR4_N,FEMALE_YR4_N,MALE_YR4_N,PELL_YR4_N,NOPELL_YR4_N,LOAN_YR4_N,NOLOAN_YR4_N,FIRSTGEN_YR4_N,NOT1STGEN_YR4_N,OVERALL_YR6_N,LO_INC_YR6_N,MD_INC_YR6_N,HI_INC_YR6_N,DEP_YR6_N,IND_YR6_N,FEMALE_YR6_N,MALE_YR6_N,PELL_YR6_N,NOPELL_YR6_N,LOAN_YR6_N,NOLOAN_YR6_N,FIRSTGEN_YR6_N,NOT1STGEN_YR6_N,OVERALL_YR8_N,LO_INC_YR8_N,MD_INC_YR8_N,HI_INC_YR8_N,DEP_YR8_N,IND_YR8_N,FEMALE_YR8_N,MALE_YR8_N,PELL_YR8_N,NOPELL_YR8_N,LOAN_YR8_N,NOLOAN_YR8_N,FIRSTGEN_YR8_N,NOT1STGEN_YR8_N,DEBT_MDN,GRAD_DEBT_MDN,WDRAW_DEBT_MDN,LO_INC_DEBT_MDN,MD_INC_DEBT_MDN,HI_INC_DEBT_MDN,DEP_DEBT_MDN,IND_DEBT_MDN,PELL_DEBT_MDN,NOPELL_DEBT_MDN,FEMALE_DEBT_MDN,MALE_DEBT_MDN,FIRSTGEN_DEBT_MDN,NOTFIRSTGEN_DEBT_MDN,DEBT_N,GRAD_DEBT_N,WDRAW_DEBT_N,LO_INC_DEBT_N,MD_INC_DEBT_N,HI_INC_DEBT_N,DEP_DEBT_N,IND_DEBT_N,PELL_DEBT_N,NOPELL_DEBT_N,FEMALE_DEBT_N,MALE_DEBT_N,FIRSTGEN_DEBT_N,NOTFIRSTGEN_DEBT_N,GRAD_DEBT_MDN10YR,CUML_DEBT_N,CUML_DEBT_P90,CUML_DEBT_P75,CUML_DEBT_P25,CUML_DEBT_P10,INC_N,DEP_INC_N,IND_INC_N,DEP_STAT_N,PAR_ED_N,APPL_SCH_N,REPAY_DT_MDN,SEPAR_DT_MDN,REPAY_DT_N,SEPAR_DT_N,RPY_1YR_N,COMPL_RPY_1YR_N,NONCOM_RPY_1YR_N,LO_INC_RPY_1YR_N,MD_INC_RPY_1YR_N,HI_INC_RPY_1YR_N,DEP_RPY_1YR_N,IND_RPY_1YR_N,PELL_RPY_1YR_N,NOPELL_RPY_1YR_N,FEMALE_RPY_1YR_N,MALE_RPY_1YR_N,FIRSTGEN_RPY_1YR_N,NOTFIRSTGEN_RPY_1YR_N,RPY_3YR_N,COMPL_RPY_3YR_N,NONCOM_RPY_3YR_N,LO_INC_RPY_3YR_N,MD_INC_RPY_3YR_N,HI_INC_RPY_3YR_N,DEP_RPY_3YR_N,IND_RPY_3YR_N,PELL_RPY_3YR_N,NOPELL_RPY_3YR_N,FEMALE_RPY_3YR_N,MALE_RPY_3YR_N,FIRSTGEN_RPY_3YR_N,NOTFIRSTGEN_RPY_3YR_N,RPY_5YR_N,COMPL_RPY_5YR_N,NONCOM_RPY_5YR_N,LO_INC_RPY_5YR_N,MD_INC_RPY_5YR_N,HI_INC_RPY_5YR_N,DEP_RPY_5YR_N,IND_RPY_5YR_N,PELL_RPY_5YR_N,NOPELL_RPY_5YR_N,FEMALE_RPY_5YR_N,MALE_RPY_5YR_N,FIRSTGEN_RPY_5YR_N,NOTFIRSTGEN_RPY_5YR_N,RPY_7YR_N,COMPL_RPY_7YR_N,NONCOM_RPY_7YR_N,LO_INC_RPY_7YR_N,MD_INC_RPY_7YR_N,HI_INC_RPY_7YR_N,DEP_RPY_7YR_N,IND_RPY_7YR_N,PELL_RPY_7YR_N,NOPELL_RPY_7YR_N,FEMALE_RPY_7YR_N,MALE_RPY_7YR_N,FIRSTGEN_RPY_7YR_N,NOTFIRSTGEN_RPY_7YR_N,COUNT_ED,LOAN_EVER,PELL_EVER,AGE_ENTRY,AGE_ENTRY_SQ,AGEGE24,FEMALE,MARRIED,DEPENDENT,VETERAN,FIRST_GEN,FAMINC,MD_FAMINC,FAMINC_IND,LNFAMINC,LNFAMINC_IND,PCT_WHITE,PCT_BLACK,PCT_ASIAN,PCT_HISPANIC,PCT_BA,PCT_GRAD_PROF,PCT_BORN_US,MEDIAN_HH_INC,POVERTY_RATE,UNEMP_RATE,LN_MEDIAN_HH_INC,FSEND_COUNT,FSEND_1,FSEND_2,FSEND_3,FSEND_4,FSEND_5,COUNT_NWNE_P10,COUNT_WNE_P10,MN_EARN_WNE_P10,MD_EARN_WNE_P10,PCT10_EARN_WNE_P10,PCT25_EARN_WNE_P10,PCT75_EARN_WNE_P10,PCT90_EARN_WNE_P10,SD_EARN_WNE_P10,COUNT_WNE_INC1_P10,COUNT_WNE_INC2_P10,COUNT_WNE_INC3_P10,COUNT_WNE_INDEP0_INC1_P10,COUNT_WNE_INDEP0_P10,COUNT_WNE_INDEP1_P10,COUNT_WNE_MALE0_P10,COUNT_WNE_MALE1_P10,GT_25K_P10,MN_EARN_WNE_INC1_P10,MN_EARN_WNE_INC2_P10,MN_EARN_WNE_INC3_P10,MN_EARN_WNE_INDEP0_INC1_P10,MN_EARN_WNE_INDEP0_P10,MN_EARN_WNE_INDEP1_P10,MN_EARN_WNE_MALE0_P10,MN_EARN_WNE_MALE1_P10,COUNT_NWNE_P6,COUNT_WNE_P6,MN_EARN_WNE_P6,MD_EARN_WNE_P6,PCT10_EARN_WNE_P6,PCT25_EARN_WNE_P6,PCT75_EARN_WNE_P6,PCT90_EARN_WNE_P6,SD_EARN_WNE_P6,COUNT_WNE_INC1_P6,COUNT_WNE_INC2_P6,COUNT_WNE_INC3_P6,COUNT_WNE_INDEP0_INC1_P6,COUNT_WNE_INDEP0_P6,COUNT_WNE_INDEP1_P6,COUNT_WNE_MALE0_P6,COUNT_WNE_MALE1_P6,GT_25K_P6,MN_EARN_WNE_INC1_P6,MN_EARN_WNE_INC2_P6,MN_EARN_WNE_INC3_P6,MN_EARN_WNE_INDEP0_INC1_P6,MN_EARN_WNE_INDEP0_P6,MN_EARN_WNE_INDEP1_P6,MN_EARN_WNE_MALE0_P6,MN_EARN_WNE_MALE1_P6,COUNT_NWNE_P7,COUNT_WNE_P7,MN_EARN_WNE_P7,SD_EARN_WNE_P7,GT_25K_P7,COUNT_NWNE_P8,COUNT_WNE_P8,MN_EARN_WNE_P8,MD_EARN_WNE_P8,PCT10_EARN_WNE_P8,PCT25_EARN_WNE_P8,PCT75_EARN_WNE_P8,PCT90_EARN_WNE_P8,SD_EARN_WNE_P8,GT_25K_P8,COUNT_NWNE_P9,COUNT_WNE_P9,MN_EARN_WNE_P9,SD_EARN_WNE_P9,GT_25K_P9,DEBT_MDN_SUPP,GRAD_DEBT_MDN_SUPP,GRAD_DEBT_MDN10YR_SUPP,RPY_3YR_RT_SUPP,LO_INC_RPY_3YR_RT_SUPP,MD_INC_RPY_3YR_RT_SUPP,HI_INC_RPY_3YR_RT_SUPP,COMPL_RPY_3YR_RT_SUPP,NONCOM_RPY_3YR_RT_SUPP,DEP_RPY_3YR_RT_SUPP,IND_RPY_3YR_RT_SUPP,PELL_RPY_3YR_RT_SUPP,NOPELL_RPY_3YR_RT_SUPP,FEMALE_RPY_3YR_RT_SUPP,MALE_RPY_3YR_RT_SUPP,FIRSTGEN_RPY_3YR_RT_SUPP,NOTFIRSTGEN_RPY_3YR_RT_SUPP,C150_L4_POOLED_SUPP,C150_4_POOLED_SUPP,C200_L4_POOLED_SUPP,C200_4_POOLED_SUPP,ALIAS,C100_4,D100_4,C100_L4,D100_L4,TRANS_4,DTRANS_4,TRANS_L4,DTRANS_L4,ICLEVEL,UGDS_MEN,UGDS_WOMEN,CDR3_DENOM,CDR2_DENOM,T4APPROVALDATE,D150_4_WHITE,D150_4_BLACK,D150_4_HISP,D150_4_ASIAN,D150_4_AIAN,D150_4_NHPI,D150_4_2MOR,D150_4_NRA,D150_4_UNKN,D150_4_WHITENH,D150_4_BLACKNH,D150_4_API,D150_4_AIANOLD,D150_4_HISPOLD,D150_L4_WHITE,D150_L4_BLACK,D150_L4_HISP,D150_L4_ASIAN,D150_L4_AIAN,D150_L4_NHPI,D150_L4_2MOR,D150_L4_NRA,D150_L4_UNKN,D150_L4_WHITENH,D150_L4_BLACKNH,D150_L4_API,D150_L4_AIANOLD,D150_L4_HISPOLD,D_PCTPELL_PCTFLOAN,OPENADMP,UGNONDS,GRADS,ACCREDCODE,OMACHT6_FTFT,OMAWDP6_FTFT,OMACHT8_FTFT,OMAWDP8_FTFT,OMENRYP8_FTFT,OMENRAP8_FTFT,OMENRUP8_FTFT,OMACHT6_PTFT,OMAWDP6_PTFT,OMACHT8_PTFT,OMAWDP8_PTFT,OMENRYP8_PTFT,OMENRAP8_PTFT,OMENRUP8_PTFT,OMACHT6_FTNFT,OMAWDP6_FTNFT,OMACHT8_FTNFT,OMAWDP8_FTNFT,OMENRYP8_FTNFT,OMENRAP8_FTNFT,OMENRUP8_FTNFT,OMACHT6_PTNFT,OMAWDP6_PTNFT,OMACHT8_PTNFT,OMAWDP8_PTNFT,OMENRYP8_PTNFT,OMENRAP8_PTNFT,OMENRUP8_PTNFT,RET_FT4_POOLED,RET_FTL4_POOLED,RET_PT4_POOLED,RET_PTL4_POOLED,RET_FT_DEN4_POOLED,RET_FT_DENL4_POOLED,RET_PT_DEN4_POOLED,RET_PT_DENL4_POOLED,POOLYRSRET_FT,POOLYRSRET_PT,RET_FT4_POOLED_SUPP,RET_FTL4_POOLED_SUPP,RET_PT4_POOLED_SUPP,RET_PTL4_POOLED_SUPP,TRANS_4_POOLED,TRANS_L4_POOLED,DTRANS_4_POOLED,DTRANS_L4_POOLED,TRANS_4_POOLED_SUPP,TRANS_L4_POOLED_SUPP,C100_4_POOLED,C100_L4_POOLED,D100_4_POOLED,D100_L4_POOLED,POOLYRS100,C100_4_POOLED_SUPP,C100_L4_POOLED_SUPP,C150_4_PELL,D150_4_PELL,C150_L4_PELL,D150_L4_PELL,C150_4_LOANNOPELL,D150_4_LOANNOPELL,C150_L4_LOANNOPELL,D150_L4_LOANNOPELL,C150_4_NOLOANNOPELL,D150_4_NOLOANNOPELL,C150_L4_NOLOANNOPELL,D150_L4_NOLOANNOPELL,GT_28K_P10,GT_28K_P8,GT_28K_P6,OMACHT6_FTFT_POOLED,OMAWDP6_FTFT_POOLED,OMACHT8_FTFT_POOLED,OMAWDP8_FTFT_POOLED,OMENRYP8_FTFT_POOLED,OMENRAP8_FTFT_POOLED,OMENRUP8_FTFT_POOLED,OMACHT6_PTFT_POOLED,OMAWDP6_PTFT_POOLED,OMACHT8_PTFT_POOLED,OMAWDP8_PTFT_POOLED,OMENRYP8_PTFT_POOLED,OMENRAP8_PTFT_POOLED,OMENRUP8_PTFT_POOLED,OMACHT6_FTNFT_POOLED,OMAWDP6_FTNFT_POOLED,OMACHT8_FTNFT_POOLED,OMAWDP8_FTNFT_POOLED,OMENRYP8_FTNFT_POOLED,OMENRAP8_FTNFT_POOLED,OMENRUP8_FTNFT_POOLED,OMACHT6_PTNFT_POOLED,OMAWDP6_PTNFT_POOLED,OMACHT8_PTNFT_POOLED,OMAWDP8_PTNFT_POOLED,OMENRYP8_PTNFT_POOLED,OMENRAP8_PTNFT_POOLED,OMENRUP8_PTNFT_POOLED,poolyrsOM_FTFT,poolyrsOM_PTFT,poolyrsOM_FTNFT,poolyrsOM_PTNFT,OMAWDP6_FTFT_POOLED_SUPP,OMAWDP8_FTFT_POOLED_SUPP,OMENRYP8_FTFT_POOLED_SUPP,OMENRAP8_FTFT_POOLED_SUPP,OMENRUP8_FTFT_POOLED_SUPP,OMAWDP6_PTFT_POOLED_SUPP,OMAWDP8_PTFT_POOLED_SUPP,OMENRYP8_PTFT_POOLED_SUPP,OMENRAP8_PTFT_POOLED_SUPP,OMENRUP8_PTFT_POOLED_SUPP,OMAWDP6_FTNFT_POOLED_SUPP,OMAWDP8_FTNFT_POOLED_SUPP,OMENRYP8_FTNFT_POOLED_SUPP,OMENRAP8_FTNFT_POOLED_SUPP,OMENRUP8_FTNFT_POOLED_SUPP,OMAWDP6_PTNFT_POOLED_SUPP,OMAWDP8_PTNFT_POOLED_SUPP,OMENRYP8_PTNFT_POOLED_SUPP,OMENRAP8_PTNFT_POOLED_SUPP,OMENRUP8_PTNFT_POOLED_SUPP'.split(',')

def parse_csv(line):
  try:
    values = line.split(',')
    rowdict = {}
    for colname, value in zip(COLNAMES, values):
      rowdict[colname] = value
    yield rowdict
  except:
    logging.warn('Ignoring line with {} values; expected {}', len(values), len(COLNAMES))
    
def pull_fields(rowdict):
  result = {}
  # required string fields (see also get_output_schema)
  for col in 'INSTNM'.split(','):
    if col in rowdict:
      result[col] = rowdict[col]
    else:
      logging.info('Ignoring line missing {}', col)
      return
      
  # float fields (see also get_output_schema)
  for col in 'ADM_RATE_ALL,FIRST_GEN,MD_FAMINC,SAT_AVG,MD_EARN_WNE_P10'.split(','):
    try:
      result[col] = (float) (rowdict[col])
    except:
      result[col] = None
  yield result
 
def get_output_schema():
  result = 'INSTNM:string'
  for col in 'ADM_RATE_ALL,FIRST_GEN,MD_FAMINC,SAT_AVG,MD_EARN_WNE_P10'.split(','):
    result = result + ',' + col + ':FLOAT64'
  print(result)
  return result  
  
def load_into_bq(infile_pattern, bqdataset='ch04', bqtable='college_scorecard_df', in_test_mode=True):
  import shutil, os, subprocess
  job_name = 'loadcsv' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
  
  if in_test_mode:
      print('Launching local job ... hang on')
      OUTPUT_DIR = './csv_load'
      shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
      os.makedirs(OUTPUT_DIR)
  else:
      print('Launching Dataflow job {} ... hang on'.format(job_name))
      OUTPUT_DIR = 'gs://{0}/csv_load'.format(BUCKET)
      try:
        subprocess.check_call('gsutil -m rm -r {}'.format(OUTPUT_DIR).split())
      except:
        pass
      
  options = {
      'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
      'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
      'job_name': job_name,
      'project': PROJECT,
      'region': REGION,
      'teardown_policy': 'TEARDOWN_ALWAYS',
      'no_save_main_session': True
  }
  opts = beam.pipeline.PipelineOptions(flags = [], **options)
  if in_test_mode:
      RUNNER = 'DirectRunner'
  else:
      RUNNER = 'DataflowRunner'
  
  with beam.Pipeline(RUNNER, options = opts) as p:
    (p 
      | 'read' >> beam.io.ReadFromText(infile_pattern, skip_header_lines=1)
      | 'parse_csv' >> beam.FlatMap(parse_csv)
      | 'pull_fields' >> beam.FlatMap(pull_fields)
      | 'write_bq' >> beam.io.gcp.bigquery.WriteToBigQuery(
          bqtable, bqdataset, schema=get_output_schema())
    )
    
load_into_bq('./college_scorecard.csv.gz', in_test_mode = False)

Launching Dataflow job loadcsv-190311-231437 ... hang on
INSTNM:string,ADM_RATE_ALL:FLOAT64,FIRST_GEN:FLOAT64,MD_FAMINC:FLOAT64,SAT_AVG:FLOAT64,MD_EARN_WNE_P10:FLOAT64


## Verify that BigQuery table exists with data

In [5]:
%%bigquery query
SELECT * from ch04.college_scorecard_df
LIMIT 5

,INSTNM,ADM_RATE_ALL,FIRST_GEN,MD_FAMINC,SAT_AVG,MD_EARN_WNE_P10
0,Lake Erie College,0.630976,0.327014,54421.0,946.0,36200.0
1,Alderson Broaddus University,0.413844,0.360544,50174.0,963.0,40400.0
2,Midstate College,0.750000,0.497653,18052.5,NaN,25600.0
3,Waubonsee Community College,NaN,0.493517,30454.0,NaN,32500.0
4,Occidental College,0.458106,0.175926,76004.0,1315.0,50600.0


Copyright Google Inc. 2019
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.